# Run and report NegDis experiments

The notebook can be run from the command line without the web interface using the command:

``` bash
jupyter nbconvert --ExecutePreprocessor.timeout=-1 --to html --output "run_cerv_experiments_`date +%Y-%m-%d`" --execute run_cerv_experiments.ipynb
```

In [ ]:
from datetime import datetime
import itertools
import json
import os
from pathlib import Path
from pprint import pprint
import re
import sys
import time

import IPython.display as nbdisplay

from negdis import aspdeclare, tools

print('Start time: ' + datetime.now().replace(microsecond=0).isoformat())

In [ ]:
DIST_DIR = Path('dist')
DATA_DIR = Path('data').joinpath('cerv')
RESULTS_DIR = Path('results').joinpath('cerv', time.strftime('%Y%m%dT%H%M%S'))
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f'Data dir: {DATA_DIR}')
print(f'Results dir: {RESULTS_DIR}')

DECL_TEMPLATES = DATA_DIR.joinpath('templates.txt')
DECL_RULES = DATA_DIR.joinpath('declare_rules.txt')

In [ ]:
tools.run_negdis('version', timeit=False, dist=DIST_DIR)
_ = aspdeclare.main(['version'])

## Show Declare parameters

In [ ]:
print(tools.cat(DECL_TEMPLATES))

In [ ]:
print(tools.cat(DECL_RULES))

## Define the set of experiments

In [ ]:
all_scenario = ['01', '02', '03', '04', '05']
ALL_EXPERIMENTS = [
    {
        'name': f'cerv_fold{sl}',
        'positive': DATA_DIR.joinpath(f'fold{sl}', f'cerv.{sl}.pos.xes'),
        'negative': DATA_DIR.joinpath(f'fold{sl}', f'cerv.{sl}.neg.xes'),
        'choices': RESULTS_DIR.joinpath(f'choices_fold{sl}.json'),
        'compatibles': RESULTS_DIR.joinpath(f'compatibles_fold{sl}.txt'),
        'optimisations': ['minclos', 'subsetclos'],
        'valpos': DATA_DIR.joinpath(f'fold{sl}', f'cerv.{sl}.valpos.xes'),
        'valneg': DATA_DIR.joinpath(f'fold{sl}', f'cerv.{sl}.valneg.xes')
    }
    for sl in all_scenario
]

ALL_EXPERIMENTS.append(
    {
        'name': 'cerv_full',
        'positive': DATA_DIR.joinpath('full', 'cerv.pos.xes'),
        'negative': DATA_DIR.joinpath('full', 'cerv.neg.xes'),
        'choices': RESULTS_DIR.joinpath('choices_full.json'),
        'compatibles': RESULTS_DIR.joinpath('compatibles_full.txt'),
        'optimisations': ['minclos', 'subsetclos'],
        'valneg': DATA_DIR.joinpath('full', 'cerv.neg.xes')
    }
)

pprint(ALL_EXPERIMENTS)

## Show log sizes

In [ ]:
for exp in ALL_EXPERIMENTS:
    for k in ('positive', 'negative', 'valpos', 'valneg'):
        fn = exp.get(k, None)
        if fn is not None:
            print('{} ({})'.format(fn, tools.xes_size(fn)))

## Generate list of compatible constraints

In [ ]:
for exp in ALL_EXPERIMENTS:
    if 'compatibles' in exp:
        pl = str(exp['positive'])
        of = str(exp['compatibles'])
        print(tools.run_negdis('discover', 'compatible', '--out', of, '-t', DECL_TEMPLATES, pl, dist=DIST_DIR))

## Generate choices files

In [ ]:
for exp in ALL_EXPERIMENTS:
    nl = exp['negative']
    pl = exp['positive']
    of = exp['choices']
    
    print('Scenario: {}'.format(exp.get('name', nl.name)))
    
    print('Discovering {},{} -> {}'.format(pl, nl, of))
    print(tools.run_negdis('discover', 'negative', '--out', of, '-t', DECL_TEMPLATES, pl, nl, dist=DIST_DIR))
    

### Display top constraints

In [ ]:
for exp in ALL_EXPERIMENTS:
    if not os.path.exists(exp['choices']):
        continue
    cf = exp['choices']
    print('-' * 60)
    print('Top constraints in {}:'.format(cf))
    print('-' * 20)
    tools.count_choices(cf)
    print('-' * 20)


## Optimisation stage

In [ ]:
MAX_MODELS = 20

for exp in ALL_EXPERIMENTS:
    if not os.path.exists(exp['choices']):
        continue
    cf = exp['choices']
    for opt_mode in exp.get('optimisations', []):
        of = cf.parent.joinpath(cf.stem + f'_opt_{opt_mode}.json')
        tools.optimise_choices(
            cf, opt_mode, DECL_RULES, timeout=60*10, models=MAX_MODELS,
            val_pos=exp.get('valpos', None), val_neg=exp.get('valneg', None),
            templates=DECL_TEMPLATES,
            dist=DIST_DIR, outdir=RESULTS_DIR, report_fp=sys.stdout)


In [ ]:
print('Finish time: ' + datetime.now().replace(microsecond=0).isoformat())


## Display machine info

In [ ]:
tools.show_hwinfo()